In [1]:
import pandas as pd
from TextHandler import *

In [2]:
sources = pd.read_json('datasets/sources.json', orient='index')

df_credible_1 = pd.read_json('datasets/source_3/scraped_articles.json')
df_credible_2 = pd.read_json('datasets/source_6/scraped_articles.json')

df_not_credible_1 = pd.read_json('datasets/source_7/scraped_articles.json')
df_not_credible_2 = pd.read_json('datasets/source_14/scraped_articles.json')

print("data_Set_1_credible:",df_credible_1.count())
print("data_Set_2_credible:",df_credible_2.count())
print("data_Set_3_not-credible:",df_not_credible_1.count())
print("data_Set_4_not-credible:",df_not_credible_2.count())

data_Set_1_credible: articles    4394
dtype: int64
data_Set_2_credible: articles    1211
dtype: int64
data_Set_3_not-credible: articles    2704
dtype: int64
data_Set_4_not-credible: articles    3381
dtype: int64


In [3]:
df_credible_3 = pd.read_json('datasets/source_1/scraped_articles.json')

df_not_credible_3 = pd.read_json('datasets/source_8/scraped_articles.json')

print("data_Set_3_credible:",df_credible_3.count())
print("data_Set_3_not-credible:",df_not_credible_3.count())


data_Set_3_credible: articles    2288
dtype: int64
data_Set_3_not-credible: articles    3654
dtype: int64


In [4]:
df_credible_1['credibility'] = 1
df_credible_2['credibility'] = 1
df_not_credible_1['credibility'] = 0
df_not_credible_2['credibility'] = 0
df_credible_3['credibility'] = 1
df_not_credible_3['credibility'] = 0

merged_df = pd.concat([df_credible_1, df_credible_2, df_not_credible_1, df_not_credible_2,df_credible_3, df_not_credible_3])

# merged_df['credibility'] = 1
# merged_df.loc[merged_df.index.isin(df_not_credible_1.index) | merged_df.index.isin(df_not_credible_2.index), 'credibility'] = 0


texts = []
for article in merged_df['articles']:
    text = article['text']
    texts.append(text)

merged_df['text'] = texts

new_df = merged_df[['text', 'credibility']]
print("New DataFrame:")
print(new_df.head())

New DataFrame:
                                                text  credibility
0  تمكنت عناصر جهاز خفر السواحل الليبي، فجر اليوم...            1
1  أكد المبعوث الأميركي الخاص إلى السودان وجنوب ا...            1
2  ‬اطلع رئيس لجنة الإدارة المكلف بشركة الخليج ال...            1
3  ‬نشرت الشركة الليبية للموانئ بيانات حديثة أظهر...            1
4  طالب حراك “الاستفتاء أولا” بعرض مسودة الدستور ...            1


In [ ]:
# df_credible_3['credibility'] = 1
# df_not_credible_3['credibility'] = 0

# merged_df_test = pd.concat([df_credible_3, df_not_credible_3])

# print(merged_df_test['credibility'].value_counts()[1])

# texts = []
# for article in merged_df_test['articles']:
#     text = article['text']
#     texts.append(text)

# merged_df_test['text'] = texts

# new_df_test = merged_df_test[['text', 'credibility']]
# print("New DataFrame:")
# print(new_df_test.head())

# data_test=new_df_test['text']
# preprocessed_data_test=preprocess_text(data_test)
# print(preprocessed_data_test[0])

In [5]:
data=new_df['text']
preprocessed_data=preprocess_text(data)
print(preprocessed_data[0])

تمك عنصر جهز خفر سحل ليب فجر اليوم قاذ هجر طرق الا شواطاء الا ورب علا متن زورق ططي نقل وكل ليب نطق رئس ارك قوت بحر ان زورق زوي تحر تلق ندء غاث عمل علا قاذ هجر زال قعد طرابلس بحر نقل الا جهز كفح هجر قنن تمم جرء رحل بلد وكان خفر سحل ليب اعد ايم قلل هجر الا ليب خلل عمل قاذ نفذ طلع اعل تحدث بسم ظمه هجر دول


In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

X_train, X_test, y_train, y_test = train_test_split(preprocessed_data, new_df['credibility'], test_size=0.2, random_state=42)

ngram_ranges = [(1, 1), (2, 2), (3, 3), (4, 4)]

classifiers = [
    LogisticRegression(),
    MultinomialNB(),
    RandomForestClassifier(),
    KNeighborsClassifier(),
    SVC()
]

metrics_dict = {}

for ngram_range in ngram_ranges:
    ngram_metrics = {}
    vectorizer = TfidfVectorizer(ngram_range=ngram_range)
    X_train_tfidf = vectorizer.fit_transform(X_train)
    X_test_tfidf = vectorizer.transform(X_test)

    X_train_normalized = preprocessing.normalize(X_train_tfidf, norm='l2')
    X_test_normalized = preprocessing.normalize(X_test_tfidf, norm='l2')

    for classifier in classifiers:
        classifier_name = classifier.__class__.__name__
        classifier_metrics = []

        classifier.fit(X_train_normalized, y_train)
        y_pred = classifier.predict(X_test_normalized)

        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        classifier_metrics.extend([accuracy, precision, recall, f1])
        ngram_metrics[classifier_name] = classifier_metrics

    metrics_dict[ngram_range] = ngram_metrics

for ngram_range, ngram_metrics in metrics_dict.items():
    print(f"n-gram range {ngram_range}:")
    for classifier_name, classifier_metrics in ngram_metrics.items():
        print(f"{classifier_name}: {classifier_metrics}")


In [12]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

X_train, X_test, y_train, y_test = train_test_split(preprocessed_data, new_df['credibility'], test_size=0.2, random_state=42)

ngram_ranges = [(1, 1), (2, 2), (3, 3), (4, 4)]

classifiers = [
    LogisticRegression(),
    MultinomialNB(),
    RandomForestClassifier(),
    KNeighborsClassifier(),
    SVC()
]

metrics_dict = {}

for ngram_range in ngram_ranges:
    ngram_metrics = {}
    vectorizer = CountVectorizer(ngram_range=ngram_range)
    Cv_train = vectorizer.fit_transform(X_train)
    Cv_test = vectorizer.transform(X_test)

    X_train_normalized = preprocessing.normalize(Cv_train, norm='l2')
    X_test_normalized = preprocessing.normalize(Cv_test, norm='l2')

    for classifier in classifiers:
        classifier_name = classifier.__class__.__name__
        classifier_metrics = []

        classifier.fit(X_train_normalized, y_train)
        y_pred = classifier.predict(X_test_normalized)

        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        classifier_metrics.extend([accuracy, precision, recall, f1])
        ngram_metrics[classifier_name] = classifier_metrics

    metrics_dict[ngram_range] = ngram_metrics

# Print the metrics for each classifier and each n-gram range
for ngram_range, ngram_metrics in metrics_dict.items():
    print(f"n-gram range {ngram_range}:")
    for classifier_name, classifier_metrics in ngram_metrics.items():
        print(f"{classifier_name}: {classifier_metrics}")


Classifier: LogisticRegression
Accuracy for n-gram range (1, 1): 0.8463283243549758
Accuracy for n-gram range (2, 2): 0.856818826197902
Accuracy for n-gram range (3, 3): 0.8432095265097816
Accuracy for n-gram range (4, 4): 0.7924581797561667
Classifier: MultinomialNB
Accuracy for n-gram range (1, 1): 0.7734618656081655
Accuracy for n-gram range (2, 2): 0.8250637935922881
Accuracy for n-gram range (3, 3): 0.8281825914374823
Accuracy for n-gram range (4, 4): 0.7862205840657783
Classifier: RandomForestClassifier
Accuracy for n-gram range (1, 1): 0.8420754182024384
Accuracy for n-gram range (2, 2): 0.8449106889707967
Accuracy for n-gram range (3, 3): 0.7785653529912107
Accuracy for n-gram range (4, 4): 0.7142047065494754
Classifier: KNeighborsClassifier
Accuracy for n-gram range (1, 1): 0.8219449957470939
Accuracy for n-gram range (2, 2): 0.8287496455911539
Accuracy for n-gram range (3, 3): 0.830450808052169
Accuracy for n-gram range (4, 4): 0.8015310462149136
Classifier: SVC
Accuracy for 

In [11]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold
from sklearn import preprocessing

X = preprocessed_data
y = np.array(new_df['credibility']) 

ngram_ranges = [(1, 1), (2, 2), (3, 3), (4, 4)]

classifiers = [
    LogisticRegression(max_iter=5000), 
    MultinomialNB(),
    KNeighborsClassifier(),
    SVC()
]

kf = KFold(n_splits=5, shuffle=True, random_state=42)

metrics_dict = {}

for classifier in classifiers:
    classifier_name = classifier.__class__.__name__
    classifier_metrics = []

    for ngram_range in ngram_ranges:
        vectorizer = TfidfVectorizer(ngram_range=ngram_range)
        X_tfidf = vectorizer.fit_transform(X)

        accuracies = []
        for train_index, test_index in kf.split(X_tfidf):
            X_train, X_test = X_tfidf[train_index], X_tfidf[test_index]
            y_train, y_test = y[train_index], y[test_index]

            scaler = preprocessing.StandardScaler(with_mean=False)
            X_train_normalized = scaler.fit_transform(X_train)
            X_test_normalized = scaler.transform(X_test)

            classifier.fit(X_train_normalized, y_train)
            y_pred = classifier.predict(X_test_normalized)
            accuracy = accuracy_score(y_test, y_pred)
            accuracies.append(accuracy)
        
        mean_accuracy = np.mean(accuracies)
        classifier_metrics.append(mean_accuracy)

    metrics_dict[classifier_name] = classifier_metrics

# Print the metrics for each classifier and each n-gram range
for classifier_name, classifier_metrics in metrics_dict.items():
    print(f"Classifier: {classifier_name}")
    for i, ngram_range in enumerate(ngram_ranges):
        print(f"Accuracy for n-gram range {ngram_range}: {classifier_metrics[i]}")


Classifier: LogisticRegression
Accuracy for n-gram range (1, 1): 0.7621369128613382
Accuracy for n-gram range (2, 2): 0.8365473799798362
Accuracy for n-gram range (3, 3): 0.8291745502364789
Accuracy for n-gram range (4, 4): 0.8044464379076505
Classifier: MultinomialNB
Accuracy for n-gram range (1, 1): 0.7608889434250103
Accuracy for n-gram range (2, 2): 0.7746143878975269
Accuracy for n-gram range (3, 3): 0.8035957119384198
Accuracy for n-gram range (4, 4): 0.7923094365948704
Classifier: KNeighborsClassifier
Accuracy for n-gram range (1, 1): 0.6049783848798854
Accuracy for n-gram range (2, 2): 0.5435605500779096
Accuracy for n-gram range (3, 3): 0.542142721708766
Accuracy for n-gram range (4, 4): 0.4875274139162423
Classifier: SVC
Accuracy for n-gram range (1, 1): 0.8025747571485251
Accuracy for n-gram range (2, 2): 0.6317479243260925
Accuracy for n-gram range (3, 3): 0.5900057429108984
Accuracy for n-gram range (4, 4): 0.5858657007983628


In [14]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import KFold
from sklearn import preprocessing

X = preprocessed_data
y = np.array(new_df['credibility']) 

ngram_ranges = [(1, 1), (2, 2), (3, 3), (4, 4)]

classifiers = [
    LogisticRegression(max_iter=5000), 
    MultinomialNB(),
    KNeighborsClassifier(),
    SVC()
]

kf = KFold(n_splits=5, shuffle=True, random_state=42)

metrics_dict = {}

for classifier in classifiers:
    classifier_name = classifier.__class__.__name__
    classifier_metrics = []

    for ngram_range in ngram_ranges:
        vectorizer = CountVectorizer(ngram_range=ngram_range)
        X_cv = vectorizer.fit_transform(X)

        accuracies = []
        for train_index, test_index in kf.split(X_cv):
            X_train, X_test = X_cv[train_index], X_cv[test_index]
            y_train, y_test = y[train_index], y[test_index]

            scaler = preprocessing.StandardScaler(with_mean=False)
            X_train_normalized = scaler.fit_transform(X_train)
            X_test_normalized = scaler.transform(X_test)

            classifier.fit(X_train_normalized, y_train)
            y_pred = classifier.predict(X_test_normalized)
            accuracy = accuracy_score(y_test, y_pred)
            accuracies.append(accuracy)
        
        mean_accuracy = np.mean(accuracies)
        classifier_metrics.append(mean_accuracy)

    metrics_dict[classifier_name] = classifier_metrics

# Print the metrics for each classifier and each n-gram range
for classifier_name, classifier_metrics in metrics_dict.items():
    print(f"Classifier: {classifier_name}")
    for i, ngram_range in enumerate(ngram_ranges):
        print(f"Accuracy for n-gram range {ngram_range}: {classifier_metrics[i]}")


Classifier: LogisticRegression
Accuracy for n-gram range (1, 1): 0.7626468434655532
Accuracy for n-gram range (2, 2): 0.8269053847790506
Accuracy for n-gram range (3, 3): 0.8223119405747832
Accuracy for n-gram range (4, 4): 0.7990583298663048
Classifier: MultinomialNB
Accuracy for n-gram range (1, 1): 0.7637814181532271
Accuracy for n-gram range (2, 2): 0.805523937292109
Accuracy for n-gram range (3, 3): 0.8150521517743119
Accuracy for n-gram range (4, 4): 0.794804796512633
Classifier: KNeighborsClassifier
Accuracy for n-gram range (1, 1): 0.6515426816000577
Accuracy for n-gram range (2, 2): 0.5353922845576166
Accuracy for n-gram range (3, 3): 0.5654486474246719
Accuracy for n-gram range (4, 4): 0.5652217453528013
Classifier: SVC
Accuracy for n-gram range (1, 1): 0.8165834070562701
Accuracy for n-gram range (2, 2): 0.6311241969212144
Accuracy for n-gram range (3, 3): 0.5855253557315971
Accuracy for n-gram range (4, 4): 0.582179285926684


In [6]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

# Check if a GPU is available and set the device accordingly
device = tf.device("cuda:0" if tf.test.is_gpu_available() else "cpu")
if tf.test.is_gpu_available():
    print("GPU")
else:
    print("CPU")
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(preprocessed_data, new_df['credibility'], test_size=0.2, random_state=42)

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

# Convert text to sequences of indices
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

# Pad sequences to ensure equal length
max_sequence_length = max(len(seq) for seq in X_train_sequences)
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_sequence_length, padding='post')
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_sequence_length, padding='post')

# Create the RNN model with LSTM cells
with device:
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=max_sequence_length),
        tf.keras.layers.LSTM(64),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Train the model
    model.fit(X_train_padded, y_train, epochs=10, batch_size=32)

    # Evaluate the model
    y_pred = model.predict_classes(X_test_padded)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy}")


Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
CPU
Epoch 1/10
110/441 [======>.......................] - ETA: 6:18 - loss: 0.6902 - accuracy: 0.5489

KeyboardInterrupt: 

In [8]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

# Check if a GPU is available and set the device accordingly
device = tf.device("cuda:0" if tf.test.is_gpu_available() else "cpu")
if tf.test.is_gpu_available():
    print("GPU")
else:
    print("CPU")
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(preprocessed_data, new_df['credibility'], test_size=0.2, random_state=42)

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

# Convert text to sequences of indices
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

# Pad sequences to ensure equal length
max_sequence_length = max(len(seq) for seq in X_train_sequences)
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_sequence_length, padding='post')
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_sequence_length, padding='post')

# Create the RNN model with GRU cells
with device:
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=max_sequence_length),
        tf.keras.layers.GRU(64),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Train the model
    model.fit(X_train_padded, y_train, epochs=10, batch_size=32)

    # Evaluate the model
    y_pred = model.predict_classes(X_test_padded)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy}")

CPU
Epoch 1/10
 12/441 [..............................] - ETA: 6:28 - loss: 0.6858 - accuracy: 0.5703

KeyboardInterrupt: 

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

# Check if a GPU is available and set the device accordingly
device = tf.device("cuda:0" if tf.test.is_gpu_available() else "cpu")
if tf.test.is_gpu_available():
    print("GPU")
else:
    print("CPU")
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(preprocessed_data, new_df['credibility'], test_size=0.2, random_state=42)

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

# Convert text to sequences of indices
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

# Pad sequences to ensure equal length
max_sequence_length = max(len(seq) for seq in X_train_sequences)
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_sequence_length, padding='post')
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_sequence_length, padding='post')

# Create the RNN model
with device:
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=max_sequence_length),
        tf.keras.layers.SimpleRNN(64),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Train the model
    model.fit(X_train_padded, y_train, epochs=10, batch_size=32)

    # Evaluate the model
    y_pred = model.predict_classes(X_test_padded)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy}")

In [ ]:
print(accuracies)
#Number of accuracies
print(len(accuracies))